In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [45]:
def scale(x_train, x_test):
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = pd.DataFrame(scaler.transform(x_train), 
                           index=x_train.index, 
                           columns=x_train.columns)

    x_test = pd.DataFrame(scaler.transform(x_test),
                          index=x_test.index, 
                          columns=x_test.columns)
    return x_train, x_test

def transform_label(y):
    labeled_y = y.apply(lambda row: int(row.damage_grade.split('_')[1])-1, 1).values
    return labeled_y

In [12]:
x = pd.read_csv('processed_data/x_post_preproc.csv', index_col='building_id')
y = pd.read_csv('processed_data/y_post_preproc.csv', index_col='building_id')

In [46]:
x_train, x_test = train_test_split(x, test_size=0.2, random_state=42)
y_train, y_test = transform_label(y.loc[x_train.index]), transform_label(y.loc[x_test.index])

In [47]:
y_train

array([3, 4, 3, ..., 3, 1, 1], dtype=int64)

In [48]:
x_train_norm, x_test_norm = scale(x_train, x_test)

In [49]:
def create_nn(lr):
    model = keras.Sequential()
    model.add(keras.layers.Dense(200, input_shape=(80,), activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(5, activation='softmax'))

    #stochastic gradient descent
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [50]:
lr=0.001
epochs=100
batch_size=100

model = create_nn(lr)

model.fit(x_train_norm, y_train, 
           epochs=epochs, batch_size=batch_size, 
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/100
609675/609675 [==============================] - 17s 28us/sample - loss: 1.3602 - accuracy: 0.4233 - val_loss: 1.1738 - val_accuracy: 0.4928
Epoch 2/100
609675/609675 [==============================] - 16s 26us/sample - loss: 1.1304 - accuracy: 0.5193 - val_loss: 1.1091 - val_accuracy: 0.5309
Epoch 3/100
609675/609675 [==============================] - 16s 26us/sample - loss: 1.0957 - accuracy: 0.5364 - val_loss: 1.0909 - val_accuracy: 0.5376
Epoch 4/100
609675/609675 [==============================] - 16s 27us/sample - loss: 1.0830 - accuracy: 0.5413 - val_loss: 1.0820 - val_accuracy: 0.5412
Epoch 5/100
609675/609675 [==============================] - 16s 27us/sample - loss: 1.0759 - accuracy: 0.5447 - val_loss: 1.0766 - val_accuracy: 0.5428
Epoch 6/100
609675/609675 [==============================] - 16s 27us/sample - loss: 1.0711 - accuracy: 0.5464 - val_loss: 1.0725 - val_accuracy: 0.5441
Epoch 7/100
609675/609675 [===

In [51]:
scores = model.evaluate(x_test_norm, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

152419/152419 [==============================] - 5s 33us/sample - loss: 1.0218 - accuracy: 0.5648s - loss: 1.0222 - accuracy: 0.56 - ETA: 1s - loss: 1.0217 

accuracy: 56.48%


In [52]:
lr=0.0001
epochs=100
batch_size=100

model1 = create_nn(lr)

model1.fit(x_train_norm, y_train, 
           epochs=epochs, batch_size=batch_size, 
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/100
609675/609675 [==============================] - 17s 27us/sample - loss: 1.5451 - accuracy: 0.3670 - val_loss: 1.4790 - val_accuracy: 0.4068
Epoch 2/100
609675/609675 [==============================] - 15s 25us/sample - loss: 1.4166 - accuracy: 0.4136 - val_loss: 1.3725 - val_accuracy: 0.4170
Epoch 3/100
609675/609675 [==============================] - 15s 25us/sample - loss: 1.3394 - accuracy: 0.4216 - val_loss: 1.3189 - val_accuracy: 0.4226
Epoch 4/100
609675/609675 [==============================] - 16s 26us/sample - loss: 1.2960 - accuracy: 0.4265 - val_loss: 1.2834 - val_accuracy: 0.4268loss: 1.3020 - accuracy: 0. - ETA: 6s - loss: 1.3022 - ac - ETA: 2s - l - ETA: 0s - loss: 1.2963 - accuracy
Epoch 5/100
609675/609675 [==============================] - 16s 26us/sample - loss: 1.2651 - accuracy: 0.4301 - val_loss: 1.2566 - val_accuracy: 0.4303 ETA: 4s - loss: 1.2690 - ac - ETA: 3s - loss: 1.2684 - ac - ETA: 3s - loss: 

Decreasing lr seems decreasing performance. Let's trying adding more layers to the NN.

In [53]:
def create_nn(lr):
    model = keras.Sequential()
    model.add(keras.layers.Dense(500, input_shape=(80,), activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(5, activation='softmax'))

    #stochastic gradient descent
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [54]:
lr=0.001
epochs=100
batch_size=100

model2 = create_nn(lr)

model2.fit(x_train_norm, y_train, 
           epochs=epochs, batch_size=batch_size, 
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/100
609675/609675 [==============================] - 20s 33us/sample - loss: 1.3088 - accuracy: 0.4410 - val_loss: 1.1748 - val_accuracy: 0.5151
Epoch 2/100
609675/609675 [==============================] - 20s 33us/sample - loss: 1.1313 - accuracy: 0.5229 - val_loss: 1.1099 - val_accuracy: 0.5271
Epoch 3/100
609675/609675 [==============================] - 20s 32us/sample - loss: 1.0975 - accuracy: 0.5340 - val_loss: 1.0928 - val_accuracy: 0.5344
Epoch 4/100
609675/609675 [==============================] - 20s 33us/sample - loss: 1.0851 - accuracy: 0.5389 - val_loss: 1.0838 - val_accuracy: 0.5378
Epoch 5/100
609675/609675 [==============================] - 20s 33us/sample - loss: 1.0776 - accuracy: 0.5426 - val_loss: 1.0780 - val_accuracy: 0.5412
Epoch 6/100
609675/609675 [==============================] - 20s 33us/sample - loss: 1.0721 - accuracy: 0.5450 - val_loss: 1.0734 - val_accuracy: 0.5429
Epoch 7/100
609675/609675 [===

In [56]:
def create_nn(lr):
    model = keras.Sequential()
    model.add(keras.layers.Dense(1000, input_shape=(80,), activation='relu'))
    model.add(keras.layers.Dense(500, activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(5, activation='softmax'))

    #stochastic gradient descent
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [57]:
lr=0.001
epochs=100
batch_size=100

model3 = create_nn(lr)

model3.fit(x_train_norm, y_train, 
           epochs=epochs, batch_size=batch_size, 
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/100
609675/609675 [==============================] - 56s 92us/sample - loss: 1.3309 - accuracy: 0.4114 - val_loss: 1.2181 - val_accuracy: 0.4482
Epoch 2/100
609675/609675 [==============================] - 57s 93us/sample - loss: 1.1635 - accuracy: 0.5022 - val_loss: 1.1285 - val_accuracy: 0.5298
Epoch 3/100
609675/609675 [==============================] - 58s 95us/sample - loss: 1.1054 - accuracy: 0.5398 - val_loss: 1.0933 - val_accuracy: 0.5434
Epoch 4/100
609675/609675 [==============================] - 60s 98us/sample - loss: 1.0823 - accuracy: 0.5470 - val_loss: 1.0787 - val_accuracy: 0.5481
Epoch 5/100
609675/609675 [==============================] - 60s 98us/sample - loss: 1.0710 - accuracy: 0.5510 - val_loss: 1.0706 - val_accuracy: 0.5503
Epoch 6/100
609675/609675 [==============================] - 60s 99us/sample - loss: 1.0630 - accuracy: 0.5538 - val_loss: 1.0644 - val_accuracy: 0.5530
Epoch 7/100
609675/609675 [===

KeyboardInterrupt: 

It stabilized around .57, which is about the same performance as model 2 (with one less layer) achieved at around epoch 60; but model 2 takes half the time to run for each epoch. Thus model 2 with 60 epochs is chosen. Model 2 has the following structure:
   - 7 layers with 500, 200, 100, 50, 20, 10 and 5 units
   
Now let's learning play with the batch size and the learning rate:

In [61]:
def create_nn(lr):
    model = keras.Sequential()
    model.add(keras.layers.Dense(500, input_shape=(80,), activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(5, activation='softmax'))

    #stochastic gradient descent
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [62]:
lr=0.001
epochs=60
batch_size=60

model2_b = create_nn(lr)

model2_b.fit(x_train_norm, y_train, 
           epochs=epochs, batch_size=batch_size,
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/60
609675/609675 [==============================] - 31s 52us/sample - loss: 1.2793 - accuracy: 0.4233 - val_loss: 1.1846 - val_accuracy: 0.4739
Epoch 2/60
609675/609675 [==============================] - 32s 53us/sample - loss: 1.1509 - accuracy: 0.5087 - val_loss: 1.1318 - val_accuracy: 0.5336
Epoch 3/60
609675/609675 [==============================] - 31s 51us/sample - loss: 1.1126 - accuracy: 0.5407 - val_loss: 1.1039 - val_accuracy: 0.5411
Epoch 4/60
609675/609675 [==============================] - 31s 51us/sample - loss: 1.0915 - accuracy: 0.5454 - val_loss: 1.0880 - val_accuracy: 0.5452
Epoch 5/60
609675/609675 [==============================] - 31s 51us/sample - loss: 1.0783 - accuracy: 0.5491 - val_loss: 1.0777 - val_accuracy: 0.5484
Epoch 6/60
609675/609675 [==============================] - 33s 54us/sample - loss: 1.0692 - accuracy: 0.5518 - val_loss: 1.0697 - val_accuracy: 0.5498
Epoch 7/60
609675/609675 [==========

With batch size = 60, it takes 40 epochs to reach .57 validation accuracy.

In [64]:
lr=0.01
epochs=40
batch_size=60

model2_c = create_nn(lr)

model2_c.fit(x_train_norm, y_train, 
           epochs=epochs, batch_size=batch_size, 
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/40
609675/609675 [==============================] - 33s 54us/sample - loss: 1.0815 - accuracy: 0.5401 - val_loss: 1.0545 - val_accuracy: 0.5505
Epoch 2/40
609675/609675 [==============================] - 32s 53us/sample - loss: 1.0404 - accuracy: 0.5592 - val_loss: 1.0378 - val_accuracy: 0.5574
Epoch 3/40
609675/609675 [==============================] - 32s 52us/sample - loss: 1.0290 - accuracy: 0.5635 - val_loss: 1.0319 - val_accuracy: 0.5604
Epoch 4/40
609675/609675 [==============================] - 30s 49us/sample - loss: 1.0213 - accuracy: 0.5667 - val_loss: 1.0258 - val_accuracy: 0.5631
Epoch 5/40
609675/609675 [==============================] - 30s 49us/sample - loss: 1.0151 - accuracy: 0.5694 - val_loss: 1.0324 - val_accuracy: 0.5612
Epoch 6/40
609675/609675 [==============================] - 33s 55us/sample - loss: 1.0101 - accuracy: 0.5711 - val_loss: 1.0232 - val_accuracy: 0.5647
Epoch 7/40
609675/609675 [==========

Decreasing the learning rate has slightly improved the validation error, let's try to decrease it further.

In [65]:
lr=0.05
epochs=40
batch_size=60

model2_c = create_nn(lr)

model2_c.fit(x_train_norm, y_train,
           epochs=epochs, batch_size=batch_size,
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/40
609675/609675 [==============================] - 32s 53us/sample - loss: 1.0587 - accuracy: 0.5502 - val_loss: 1.0538 - val_accuracy: 0.5495
Epoch 2/40
609675/609675 [==============================] - 35s 57us/sample - loss: 1.0317 - accuracy: 0.5622 - val_loss: 1.0398 - val_accuracy: 0.5573
Epoch 3/40
609675/609675 [==============================] - 31s 51us/sample - loss: 1.0206 - accuracy: 0.5664 - val_loss: 1.0273 - val_accuracy: 0.5620
Epoch 4/40
609675/609675 [==============================] - 33s 54us/sample - loss: 1.0128 - accuracy: 0.5698 - val_loss: 1.0198 - val_accuracy: 0.5652
Epoch 5/40
609675/609675 [==============================] - 33s 54us/sample - loss: 1.0063 - accuracy: 0.5716 - val_loss: 1.0212 - val_accuracy: 0.5648
Epoch 6/40
609675/609675 [==============================] - 36s 60us/sample - loss: 1.0006 - accuracy: 0.5745 - val_loss: 1.0126 - val_accuracy: 0.5675
Epoch 7/40
609675/609675 [==========

It stabilizes, moving between .57 and a bit over .575 after around 10 epochs.

In [66]:
lr=0.1
epochs=20
batch_size=60

model2_e = create_nn(lr)

model2_e.fit(x_train_norm, y_train,
           epochs=epochs, batch_size=batch_size,
           validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/20
609675/609675 [==============================] - 38s 62us/sample - loss: 1.0596 - accuracy: 0.5509 - val_loss: 1.0442 - val_accuracy: 0.5533
Epoch 2/20
609675/609675 [==============================] - 40s 65us/sample - loss: 1.0326 - accuracy: 0.5616 - val_loss: 1.0272 - val_accuracy: 0.5626
Epoch 3/20
609675/609675 [==============================] - 38s 63us/sample - loss: 1.0216 - accuracy: 0.5660 - val_loss: 1.0250 - val_accuracy: 0.5627
Epoch 4/20
609675/609675 [==============================] - 38s 62us/sample - loss: 1.0134 - accuracy: 0.5698 - val_loss: 1.0284 - val_accuracy: 0.5610
Epoch 5/20
609675/609675 [==============================] - 31s 51us/sample - loss: 1.0068 - accuracy: 0.5729 - val_loss: 1.0166 - val_accuracy: 0.5672
Epoch 6/20
609675/609675 [==============================] - 34s 55us/sample - loss: 1.0014 - accuracy: 0.5748 - val_loss: 1.0096 - val_accuracy: 0.5710
Epoch 7/20
609675/609675 [==========

Performance decreased, so the final choice is model 2 with:
- learning rate = 0.05
- epochs = 15
- batch_size=60